# Useful tools 

In [18]:
from IPython.html import widgets
from IPython.html.widgets import interact
from IPython.display import display

import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from glob import glob
import os

In [19]:
def camera_cal(cal_dir, nx, ny):
    """ camera calibration based on multiple chessboard images
    https://docs.opencv.org/3.4.3/dc/dbb/tutorial_py_calibration.html
    input:
        cal_dir: input image directory 
        nx: number of corners in a row 
        ny: number of corners in a column

    return:
        mtx: camera matrix
        dist: distortion coefficients

    """
    img_paths = glob(cal_dir + '/calibration*.jpg')

    # print(img_paths)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.

    objp = np.zeros((nx * ny,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

    for path in img_paths:
        img = cv2.imread(path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)

        if ret == True:
            cv2.drawChessboardCorners(img, (nx,ny), corners, ret)
            objpoints.append(objp)
            imgpoints.append(corners)

            # cv2.imshow('img', img)
            # cv2.waitKey(500)

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    return (ret, mtx, dist, rvecs, tvecs)

In [20]:
def hls_thres_L(img, thresh=(0, 255)):
    """ thresholding based on L channel value of HLS image 
    Input:
        img: BGR image read by cv2
        thresh: low and upper thresholds for values of L channel
    Output:
        binary_output: binary map of preserved values in-between the thresholds
    """
    img_HLS = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    L_channel = img_HLS[...,-2]
    binary_output = np.zeros_like(L_channel)
    binary_output[(L_channel>thresh[0])&(L_channel<=thresh[1])] = 1
    
    return binary_output

In [21]:
def hls_thres_S(img, thresh=(0, 255)):
    """ thresholding based on S channel value of HLS image 
    Input:
        img: BGR image read by cv2
        thresh: low and upper thresholds for values of S channel
    Output:
        binary_output: binary map of preserved values in-between the thresholds
    """

    
    img_HLS = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    S_channel = img_HLS[...,-1]
    binary_output = np.zeros_like(S_channel)
    binary_output[(S_channel>thresh[0])&(S_channel<=thresh[1])] = 1
    
    return binary_output

In [22]:
def dir_threshold(image, sobel_kernel=3, thresh=(0, np.pi/2)):
    """ thresholding method based on direction of derivatives
    Input: 
        image: BGR image read by cv2
        sobel_kernel: size of sobel kernel
        thresh: low and upper thresholds for direction of derivatives
    Output:
        binary_output: binary map of preserved values in-between the thresholds
    """

    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    img_gray_x_deriv = np.absolute(cv2.Sobel(img_gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    img_gray_y_deriv = np.absolute(cv2.Sobel(img_gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    
    grad_dir = np.arctan2(img_gray_y_deriv, img_gray_x_deriv)
    
    binary_output = np.zeros_like(grad_dir)
    
    binary_output[(grad_dir>thresh[0])&(grad_dir<thresh[1])] = 1
    

    return binary_output

In [23]:
def mag_thresh(image, sobel_kernel=3, mag_thresh=(0, 255)):
    """ thresholding method absed on magnitude of derivatives
    Input:
        image: BGR image read by cv2
        sobel_kernel: size of sobel kernel
        mag_thresh: low and upper thresholds for magnitudes of derivatives
    Output:
        binary_output: binary map of preserved values in-between the thresholds
    """

    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    img_gray_x_deriv = cv2.Sobel(img_gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    img_gray_y_deriv = cv2.Sobel(img_gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    img_gray_xy_deriv = np.sqrt(img_gray_x_deriv**2 + img_gray_y_deriv**2)
    
    img_gray_xy_deriv = np.uint8(255*np.absolute(img_gray_xy_deriv)/np.max(np.absolute(img_gray_xy_deriv)))
    
    binary_output = np.zeros_like(img_gray_xy_deriv)
    
    
    binary_output[(img_gray_xy_deriv>mag_thresh[0])&(img_gray_xy_deriv<mag_thresh[1])] = 1
    
    return binary_output

In [24]:
def combined_binary_mask(image, ksize, mag_low, mag_high, dir_low, dir_high, hls_low, hls_high, bright_low, bright_high):
    
    S_bin = hls_thres_S(image, (hls_low, hls_high))
    L_bin = hls_thres_L(image, (bright_low, bright_high))
    mag_bin = mag_thresh(image, ksize, (mag_low, mag_high))
    dir_bin = dir_threshold(image, ksize, (dir_low, dir_high))
    
    combined = np.zeros_like(S_bin)
    
    S_ = S_bin == 1
    L_ = L_bin == 1
    mag_ = mag_bin == 1
    dir_ = dir_bin == 1
    
    combined[(S_ | L_) & (mag_ & dir_)] = 1
    
    return combined

In [25]:
def interactive_mask(ksize, mag_low, mag_high, dir_low, dir_high, hls_low, hls_high, bright_low, bright_high):
    combined = combined_binary_mask(image, ksize, mag_low, mag_high, dir_low, dir_high,\
                                    hls_low, hls_high, bright_low, bright_high)
    plt.figure(figsize=(10,10))
    plt.imshow(combined,cmap='gray')


In [26]:

image_path = './challenge_images/image_140.png'

img_dir = './camera_cal'

_, mtx, dist, _, _ = camera_cal(img_dir, nx=9, ny=6)
    
image = cv2.imread(image_path)
image = cv2.undistort(image, mtx, dist, None, mtx)


interact(interactive_mask, ksize=(1,31,2), mag_low=(0,255), mag_high=(0,255),\
         dir_low=(0, np.pi/2), dir_high=(0, np.pi/2), hls_low=(0,255),\
         hls_high=(0,255), bright_low=(0,255), bright_high=(0,255))

interactive(children=(IntSlider(value=15, description='ksize', max=31, min=1, step=2), IntSlider(value=127, de…

<function __main__.interactive_mask(ksize, mag_low, mag_high, dir_low, dir_high, hls_low, hls_high, bright_low, bright_high)>